In [1]:
import tkinter as tk

In [16]:
root = tk.Tk()

entry_var = tk.StringVar()
label = tk.Label(root, text='Color Value: ')
entry1 = tk.Entry(root, textvariable = entry_var, font=('sarif',12))
label1 = tk.Label(root, width = 50, height = 10, bg='white')
button1 = tk.Button(root, text='Set Color')

xpad = 10

label.place(x=10,y=10)
entry1.pack(pady=(10,5), padx = xpad)
label1.pack(pady=5,padx = xpad)
button1.pack(pady=5, padx = xpad)


def get_color():
    print("selected color: ",entry_var.get())
    label1.config(bg=entry_var.get())

    
button1.config(command = get_color)
root.mainloop()

In [31]:
import tkinter as tk
from tkinter import ttk, font
root = tk.Tk()

entry_var = tk.StringVar()

font_list = font.families(root)
entry_var.set(font_list[0])


text = "Almost before we knew it, we had left the ground."

label = tk.Label(root, text='Color Value: ')
entry1 = ttk.Combobox(root, textvariable = entry_var, values=font_list )
label1 = tk.Message(root, text=text , aspect = 400, font = ('sarif', 20))


xpad = 10

label.place(x=10,y=10)
entry1.pack(pady=(10,5), padx = xpad)
label1.pack(pady=5,padx = xpad)

def get_text(event):
    print(entry_var.get())
    label1.config(font = (entry_var.get(),20))
    print(event)
    
entry1.bind("<<ComboboxSelected>>", get_text)

root.mainloop()

Fixedsys
<VirtualEvent event x=0 y=0>
Terminal
<VirtualEvent event x=0 y=0>
System
<VirtualEvent event x=0 y=0>
8514oem
<VirtualEvent event x=0 y=0>
Fixedsys
<VirtualEvent event x=0 y=0>


### Testing of ttk.Checkbutton label text overstrike

In [37]:
import tkinter as tk
from tkinter import ttk, font

In [55]:
# Now we need make some checkbutton 

root = tk.Tk()

normal = font.Font(root, family='sarif', size=12)
overstriked = font.Font(root, family='sarif', size=12, overstrike=1)

style = ttk.Style()
style.configure('UnCheck.TCheckbutton', font=normal)
style.configure("Check.TCheckbutton", font=overstriked)


textvar = tk.StringVar()
textvar.set('Hello this is test trial!')
var = tk.IntVar()
var.set(0)


check  = ttk.Checkbutton(root, textvariable = textvar, variable=var, style = 'UnCheck.TCheckbutton')
check.pack(pady=50)

def oncheck(event):
    result =var.get() 

    if not result:  # selected
        print("selected")
        check.configure(style = "Check.TCheckbutton")
    else:   # Not selected.
        print("deselected!")
        check.configure(style= "UnCheck.TCheckbutton")

check.bind("<Button-1>", oncheck)
root.mainloop()

selected
deselected!
selected
deselected!
selected
deselected!
selected
deselected!


#### tkinter ttk frame test

In [3]:
import tkinter as tk
from tkinter import ttk

In [5]:
root = tk.Tk()

style= ttk.Style()


frame = ttk.Frame(root, width=250, height=300)
frame.pack()
print(frame.winfo_class())
root.mainloop()

TFrame


## Watcher modification in the dict 

In [ ]:
"""Observer descriptor class allows to trigger out any arbitrary action, when the content of observed
data changes.
"""

import weakref


class Observer(object):
    """Observes attached data and trigger out given action if the content of data changes.
    Observer is a descriptor, which means, it must be declared on the class definition level.

    Example:
        >>> def action(observer, instance, value):
        ...     print 'Data has been modified: %s' % value

        >>> class MyClass(object):
        ...     important_data = Observer('init_value', callback=action)

        >>> o = MyClass()
        >>> o.important_data = 'new_value'
        Data has been modified: new_value


    Observer should work with any kind of built-in data types, but `dict` and `list` are strongly advice.

    Example:
        >>> class MyClass2(object):
        ...     important_data = Observer({}, callback=action)
        >>> o2 = MyClass2()
        >>> o2.important_data['key1'] = {'item1': 'value1', 'item2': 'value2'}
        Data has been modified: {'key1': {'item2': 'value2', 'item1': 'value1'}}
        >>> o2.important_data['key1']['item1'] = range(5)
        Data has been modified: {'key1': {'item2': 'value2', 'item1': [0, 1, 2, 3, 4]}}
        >>> o2.important_data['key1']['item1'][0] = 'first'
        Data has been modified: {'key1': {'item2': 'value2', 'item1': ['first', 1, 2, 3, 4]}}


    Here is an example of using `Observer` as a base class.

    Example:
        >>> class AdvanceDescriptor(Observer):
        ...     def action(self, instance, value):
        ...         logger = instance.get_logger()
        ...         logger.info(value)
        ...
        ...     def __init__(self, additional_data=None, **kwargs):
        ...         self.additional_data = additional_data
        ...
        ...         super(AdvanceDescriptor, self).__init__(
        ...             callback=AdvanceDescriptor.action,
        ...             init_value={},
        ...             additional_data=additional_data
        ...         )
    """

    def __init__(self, init_value=None, callback=None, **kwargs):
        """
        Args:
            init_value: initial value for data, if there is none
            callback: callback function to evoke when the content of data will change; the signature of
                the callback should be callback(observer, instance, value), where:
                    observer is an Observer object, with all additional data attached to it,
                    instance is an instance of the object, where the actual data lives,
                    value is the data itself.
            **kwargs: additional arguments needed to make inheritance possible. See the example above, to get an
                idea, how the proper inheritance should look like.
                The main challenge here comes from the fact, that class constructor is used inside the class methods,
                which is quite tricky, when you want to change the `__init__` function signature in derived classes.
        """
        self.init_value = init_value
        self.callback = callback
        self.kwargs = kwargs
        self.kwargs.update({
            'callback': callback,
        })

        self._value = None

        self._instance_to_name_mapping = {}
        self._instance = None

        self._parent_observer = None

        self._value_parent = None
        self._value_index = None

    @property
    def value(self):
        """Returns the content of attached data.
        """
        return self._value

    def _get_attr_name(self, instance):
        """To respect DRY methodology, we try to find out, what the original name of the descriptor is and
        use it as instance variable to store actual data.

        Args:
            instance: instance of the object

        Returns: (str): attribute name, where `Observer` will store the data
        """
        if instance in self._instance_to_name_mapping:
            return self._instance_to_name_mapping[instance]
        for attr_name, attr_value in instance.__class__.__dict__.iteritems():
            if attr_value is self:
                self._instance_to_name_mapping[weakref.ref(instance)] = attr_name
                return attr_name

    def __get__(self, instance, owner):
        attr_name = self._get_attr_name(instance)
        attr_value = instance.__dict__.get(attr_name, self.init_value)

        observer = self.__class__(**self.kwargs)
        observer._value = attr_value
        observer._instance = instance
        return observer

    def __set__(self, instance, value):
        attr_name = self._get_attr_name(instance)
        instance.__dict__[attr_name] = value
        self._value = value
        self._instance = instance
        self.divulge()

    def __getitem__(self, key):
        observer = self.__class__(**self.kwargs)
        observer._value = self._value[key]
        observer._parent_observer = self
        observer._value_parent = self._value
        observer._value_index = key
        return observer

    def __setitem__(self, key, value):
        self._value[key] = value
        self.divulge()

    def divulge(self):
        """Divulges that data content has been change calling callback.
        """
        # we want to evoke the very first observer with complete set of data, not the nested one
        if self._parent_observer:
            self._parent_observer.divulge()
        else:
            if self.callback:
                self.callback(self, self._instance, self._value)

    def __getattr__(self, item):
        """Mock behaviour of data attach to `Observer`. If certain behaviour mutate attached data, additional
        wrapper comes into play, evoking attached callback.
        """

        def observe(o, f):
            def wrapper(*args, **kwargs):
                result = f(*args, **kwargs)
                o.divulge()
                return result

            return wrapper

        attr = getattr(self._value, item)

        if item in (
                    ['append', 'extend', 'insert', 'remove', 'pop', 'sort', 'reverse'] + # list methods
                    ['clear', 'pop', 'update']                                           # dict methods
        ):
            return observe(self, attr)
        return attr


def action(self, instance, value):
    print '>> log >', value, '<<'


class MyClass(object):
    meta = Observer('', action)


mc1 = MyClass()
mc2 = MyClass()

mc1.meta = {
    'a1': {
        'a11': 'a11_val',
        'a22': 'a22_val',
    },
    'b1': 'val_b1',
}
mc1.meta['a1']['a11'] = ['1', '2', '4']
mc1.meta['a1']['a11'].append('5')
mc1.meta.update({'new': 'new_value'})

mc2.meta = 'test'
mc2.meta = 'test2'
mc2.meta = range(10)
mc2.meta[5] = 'test3'
mc2.meta[9] = {
    'a': 'va1',
    'b': 'va2',
    'c': 'va3',
    'd': 'va4',
    'e': 'va5',
}
mc2.meta[9]['a'] = 'val1_new'


class MyClass2(object):
    pkg = Observer('', action)


mc3 = MyClass2()
mc3.pkg = 'test_myclass2'
print mc1.meta.value